<a href="https://colab.research.google.com/github/Effervescebt/Computer-System-Design/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


Fri Aug  8 06:16:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip -q install "transformers>=4.42.0" accelerate bitsandbytes pillow torchvision tqdm huggingface_hub



In [ ]:
from google.colab import drive, files
import os
drive.mount('/content/drive')

HF_HOME   = "/content/drive/MyDrive/hf_cache"
MODEL_DIR = "/content/drive/MyDrive/models/llava-1.5-7b-hf"
DATA_DIR  = "/content/subset_vqa_2000"

os.makedirs(HF_HOME, exist_ok=True); os.environ["HF_HOME"] = HF_HOME
os.makedirs(MODEL_DIR, exist_ok=True)


Mounted at /content/drive


In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id="llava-hf/llava-1.5-7b-hf",
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

'/content/drive/MyDrive/models/llava-1.5-7b-hf'

In [ ]:
%%writefile /content/run_llava_infer.py
# -*- coding: utf-8 -*-
import os, json, argparse
from PIL import Image
from tqdm import tqdm
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--in_jsonl", required=True)
    ap.add_argument("--out_preds", required=True)
    ap.add_argument("--model_id", default="llava-hf/llava-1.5-7b-hf")
    ap.add_argument("--max_new_tokens", type=int, default=32)
    ap.add_argument("--load_in_4bit", action="store_true")
    args = ap.parse_args()

    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model = LlavaForConditionalGeneration.from_pretrained(
        args.model_id,
        torch_dtype=torch_dtype,
        device_map="auto",
        low_cpu_mem_usage=True,
        load_in_4bit=args.load_in_4bit
    )
    processor = AutoProcessor.from_pretrained(args.model_id)

    root = os.path.dirname(args.in_jsonl)
    preds = []
    with open(args.in_jsonl, "r", encoding="utf-8") as f:
        lines = [json.loads(x) for x in f]

    for r in tqdm(lines, total=len(lines)):
        qid = int(r["question_id"])
        img_path = r["image"]
        if not os.path.isabs(img_path):
            img_path = os.path.join(root, img_path)
        image = Image.open(img_path).convert("RGB")
        question = r["question"]

        convo = [{"role":"user","content":[{"type":"image"}, {"type":"text","text":question}]}]
        prompt = processor.apply_chat_template(convo, add_generation_prompt=True)
        inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=args.max_new_tokens, do_sample=False)
        text = processor.batch_decode(out, skip_special_tokens=True)[0]
        ans = text.split("ASSISTANT:")[-1].strip()
        preds.append({"question_id": qid, "answer": ans})

    os.makedirs(os.path.dirname(args.out_preds), exist_ok=True)
    with open(args.out_preds, "w", encoding="utf-8") as f:
        json.dump(preds, f)
    print("Saved:", args.out_preds)

if __name__ == "__main__":
    main()


Writing /content/run_llava_infer.py


In [ ]:
%%writefile /content/score_preds.py
# -*- coding: utf-8 -*-
import json, re, argparse, csv, collections
NUM_WORDS = {"zero":"0","one":"1","two":"2","three":"3","four":"4","five":"5","six":"6","seven":"7","eight":"8","nine":"9","ten":"10","eleven":"11","twelve":"12","thirteen":"13","fourteen":"14","fifteen":"15","sixteen":"16","seventeen":"17","eighteen":"18","nineteen":"19","twenty":"20"}
ARTICLES = {"a","an","the"}
def normalize(a):
    a = a.strip().lower(); a = re.sub(r"[^\w\s]", " ", a)
    toks = [w for w in a.split() if w not in ARTICLES]
    toks = [NUM_WORDS.get(w, w) for w in toks]
    return re.sub(r"\s+"," "," ".join(toks)).strip()
def score(pred_norm, answers):
    m = sum(1 for x in answers if normalize(x.get("answer","")) == pred_norm)
    return min(1.0, m/3.0)
def load_preds(p):
    obj = json.load(open(p,"r",encoding="utf-8"))
    return ({int(x["question_id"]): x["answer"] for x in obj} if isinstance(obj,list)
            else {int(k): v for k,v in obj.items()})
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--mini_a", required=True)
    ap.add_argument("--preds", required=True)
    ap.add_argument("--report_csv", default=None)
    a = ap.parse_args()
    ann = json.load(open(a.mini_a,"r",encoding="utf-8"))
    ann_by_qid = {x["question_id"]: x for x in ann["annotations"]}
    preds = load_preds(a.preds)
    tot,s = 0,0.0; bytype = collections.defaultdict(lambda: {"n":0,"s":0.0})
    for qid, ans in preds.items():
        if qid not in ann_by_qid: continue
        acc = score(normalize(ans), ann_by_qid[qid].get("answers",[]))
        s += acc; tot += 1
        t = ann_by_qid[qid].get("answer_type","other")
        bytype[t]["n"] += 1; bytype[t]["s"] += acc
    overall = s/tot if tot else 0.0
    print(f"samples={tot}  overall_acc={overall:.4f}")
    for k,v in bytype.items():
        print(f"acc_{k}={(v['s']/v['n'] if v['n'] else 0.0):.4f}")
    if a.report_csv:
        row = {"overall_acc": overall, "samples": tot}
        for k,v in bytype.items(): row[f"acc_{k}"] = v["s"]/v["n"] if v["n"] else 0.0
        with open(a.report_csv,"w",newline="",encoding="utf-8") as f:
            import csv; w = csv.DictWriter(f, fieldnames=row.keys()); w.writeheader(); w.writerow(row)
if __name__ == "__main__":
    main()


Writing /content/score_preds.py


In [ ]:
%%writefile /content/run_llava_infer.py
# -*- coding: utf-8 -*-
import os, json, argparse, sys
from PIL import Image
from tqdm import tqdm
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--in_jsonl", required=True)
    ap.add_argument("--out_preds", required=True)
    ap.add_argument("--model_id", default="llava-hf/llava-1.5-7b-hf")
    ap.add_argument("--max_new_tokens", type=int, default=12)  # VQA 短答足够
    ap.add_argument("--load_in_4bit", action="store_true")
    ap.add_argument("--save_every", type=int, default=200, help="flush every N samples")
    args = ap.parse_args()

    # ===== 4bit 量化（新写法） =====
    quantization_config = None
    if args.load_in_4bit:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,  # 比 float32 更快
        )

    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model = LlavaForConditionalGeneration.from_pretrained(
        args.model_id,
        device_map="auto",
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        quantization_config=quantization_config,  # 用新接口，不再传 load_in_4bit
    )
    processor = AutoProcessor.from_pretrained(args.model_id, use_fast=True)
    model.eval()
    torch.set_grad_enabled(False)

    # ===== 断点续跑：读取已完成的预测 =====
    os.makedirs(os.path.dirname(args.out_preds), exist_ok=True)
    tmp_jsonl = args.out_preds + ".tmp.jsonl"   # 流式增量文件
    pred_list = []
    done_qids = set()

    if os.path.exists(tmp_jsonl):
        with open(tmp_jsonl, "r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                rec = json.loads(line)
                pred_list.append(rec)
                qid = int(rec.get("question_id"))
                done_qids.add(qid)
        print(f"[resume] loaded {len(pred_list)} records from {tmp_jsonl}", flush=True)

    fout = open(tmp_jsonl, "a", encoding="utf-8")

    def flush_snapshot():
        """把 list 快照保存到 out_preds（你的评分脚本可直接用）"""
        snap = args.out_preds + ".partial"
        with open(snap, "w", encoding="utf-8") as f:
            json.dump(pred_list, f, ensure_ascii=False)
        os.replace(snap, args.out_preds)
        fout.flush()
        try:
            os.fsync(fout.fileno())
        except Exception:
            pass

    # ===== 读取输入 =====
    root = os.path.dirname(args.in_jsonl)
    with open(args.in_jsonl, "r", encoding="utf-8") as f:
        lines = [json.loads(x) for x in f if x.strip()]

    # ===== 生成参数：保守解码，稳定短答 =====
    gen_kwargs = dict(
        do_sample=False,
        temperature=0.0,
        top_p=1.0,
        num_beams=1,
        max_new_tokens=args.max_new_tokens,
        repetition_penalty=1.0,
    )

    pbar = tqdm(lines, total=len(lines), ncols=0)
    for r in pbar:
        qid = int(r["question_id"])
        if qid in done_qids:
            continue

        img_path = r["image"]
        if not os.path.isabs(img_path):
            img_path = os.path.join(root, img_path)

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # 若图片损坏/缺失，给个占位答案，避免整个进程中断
            pred_list.append({"question_id": qid, "answer": ""})
            fout.write(json.dumps({"question_id": qid, "answer": ""}, ensure_ascii=False) + "\n")
            if len(pred_list) % max(1, args.save_every) == 0:
                flush_snapshot()
            continue

        question = r["question"]

        # LLaVA 对话模板
        convo = [{"role":"user","content":[{"type":"image"}, {"type":"text","text":question}]}]
        prompt = processor.apply_chat_template(convo, add_generation_prompt=True)

        inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(**inputs, **gen_kwargs)

        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        # 和你原来一致：截取 ASSISTANT: 之后
        ans = text.split("ASSISTANT:")[-1].strip()

        rec = {"question_id": qid, "answer": ans}
        pred_list.append(rec)
        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")

        if len(pred_list) % max(1, args.save_every) == 0:
            pbar.set_description(f"flushing @ {len(pred_list)}")
            flush_snapshot()

    # 最后一次完整保存
    flush_snapshot()
    fout.close()
    print("Saved:", args.out_preds)

if __name__ == "__main__":
    main()


Overwriting /content/run_llava_infer.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"

!ls -lah "{DATA_DIR}"

!head -n 5 "{DATA_DIR}/inference.jsonl" > "{DATA_DIR}/inference_5.jsonl"

!python /content/run_llava_infer.py \
  --in_jsonl "{DATA_DIR}/inference_5.jsonl" \
  --out_preds "{DATA_DIR}/my_preds_5.json" \
  --model_id "/content/drive/MyDrive/models/llava-1.5-7b-hf" \
  --load_in_4bit

!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_5.json"


total 2.3M
drwx------ 2 root root 4.0K Aug  8 00:00 images
-rw------- 1 root root 240K Aug  7 23:10 inference.jsonl
-rw------- 1 root root 319K Aug  7 23:10 manifest.csv
-rw------- 1 root root 1.6M Aug  7 22:47 mini_val_annotations.json
-rw------- 1 root root 185K Aug  7 22:47 mini_val_questions.json
2025-08-08 00:25:53.139719: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-08 00:25:53.156914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754612753.177720   15087 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754612

In [ ]:
import json, os
DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
ann_path = f"{DATA_DIR}/mini_val_annotations.json"
with open(ann_path, "r", encoding="utf-8") as f:
    ann = json.load(f)

print(type(ann))
if isinstance(ann, dict):
    print("top-level keys:", list(ann.keys())[:10])
    items = ann.get("annotations") or ann.get("data") or []
elif isinstance(ann, list):
    items = ann
else:
    items = []

print("num items:", len(items))
print("sample keys:", list(items[0].keys()) if items else "EMPTY")
# 看第一条里有没有 "answers"、"multiple_choice_answer"
if items:
    print("has answers:", "answers" in items[0])
    print("has mca:", "multiple_choice_answer" in items[0])


<class 'dict'>
top-level keys: ['info', 'license', 'data_subtype', 'annotations', 'data_type']
num items: 2000
sample keys: ['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id']
has answers: True
has mca: True


In [ ]:
import json, re, collections

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
ann = json.load(open(f"{DATA_DIR}/mini_val_annotations.json","r",encoding="utf-8"))
preds = json.load(open(f"{DATA_DIR}/my_preds_5.json","r",encoding="utf-8"))

NUM = {"zero":"0","one":"1","two":"2","three":"3","four":"4","five":"5","six":"6","seven":"7","eight":"8","nine":"9","ten":"10"}
ART = {"a","an","the"}

def norm(a:str):
    a = a.strip().lower()
    a = re.sub(r"[^0-9a-z\s]", " ", a)
    a = re.sub(r"\b(" + "|".join(ART) + r")\b", " ", a)
    a = " ".join(NUM.get(w,w) for w in a.split())
    # 额外兼容：常见短语化→单词化
    a = re.sub(r"^to\s+", "", a)           # "to right" → "right"
    a = re.sub(r"\b(on|at|in)\s+", "", a)  # "on right" → "right"
    a = re.sub(r"\s+side$", "", a)         # "right side" → "right"
    return a.strip()

items = ann["annotations"] if isinstance(ann, dict) else ann
ann_by_qid = {int(x["question_id"]): x for x in items}
if isinstance(preds, dict):
    preds = [{"question_id": int(k), "answer": v} for k,v in preds.items()]

def vqa_score(pred, answers):
    n = sum(1 for a in answers if norm(a.get("answer","")) == norm(pred))
    return min(1.0, n/3.0)

bad = []
for p in preds:
    qid = int(p["question_id"])
    gt = ann_by_qid.get(qid)
    if not gt:
        print("⚠️ qid不在标注里:", qid);
        continue
    ans_list = gt.get("answers", [])
    if not ans_list and gt.get("multiple_choice_answer"):
        ans_list = [{"answer": gt["multiple_choice_answer"]} for _ in range(10)]
    pred_raw = str(p["answer"])
    sc = vqa_score(pred_raw, ans_list)
    gts = [a["answer"] for a in ans_list]
    print(f"qid={qid}  score={sc:.3f}  pred='{pred_raw}'  pred_norm='{norm(pred_raw)}'")
    if sc == 0:
        # 展示最常见的 GT（有些题10个标注里会重复）
        c = collections.Counter(norm(a) for a in gts)
        print("  GT top:", c.most_common(3))
        bad.append(qid)

print("----")
print("总数:", len(preds), "  0分样本:", len(bad))


qid=363652002  score=0.000  pred='The fire fighting implement in the image is a fire hydrant.'  pred_norm='fire fighting implement image is fire hydrant'
  GT top: [('hydrant', 7), ('fire hydrant', 3)]
qid=278449003  score=0.000  pred='The pizza is round in shape.'  pred_norm='pizza is round shape'
  GT top: [('oval', 4), ('round', 3), ('roundish', 1)]
qid=396676005  score=0.000  pred='The arrow points to the right.'  pred_norm='arrow points to right'
  GT top: [('up', 8), ('forward', 1), ('straight', 1)]
qid=381832001  score=0.000  pred='The streetlamp is red.'  pred_norm='streetlamp is red'
  GT top: [('red', 4), ('yellow', 3), ('silver', 2)]
qid=302964001  score=0.000  pred='This is a large red passenger bus.'  pred_norm='this is large red passenger bus'
  GT top: [('bus', 10)]
----
总数: 5   0分样本: 5


In [ ]:
import json, os, itertools

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
sub = os.path.join(DATA_DIR, "inference_5.jsonl")  # 你刚才生成的那个
missing = []
samples = []
with open(sub, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if not line.strip():
            continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path") or obj.get("img")  # 尝试常见字段
        qid = obj.get("question_id")
        samples.append((qid, img))
        # 相对路径 -> 以数据目录为基准
        abs_img = img if img.startswith("/") else os.path.join(DATA_DIR, img)
        if not os.path.exists(abs_img):
            missing.append((qid, img, abs_img))

print("Samples:", samples)
print("Missing:", missing)


Samples: [(363652002, 'images/COCO_val2014_000000363652.jpg'), (278449003, 'images/COCO_val2014_000000278449.jpg'), (396676005, 'images/COCO_val2014_000000396676.jpg'), (381832001, 'images/COCO_val2014_000000381832.jpg'), (302964001, 'images/COCO_val2014_000000302964.jpg')]
Missing: []


In [ ]:
import json, itertools, os
pred_path = "/content/drive/MyDrive/baseline_test/subset_vqa_2000/my_preds_5.json"
with open(pred_path, "r", encoding="utf-8") as f:
    preds = json.load(f)

print("Type:", type(preds))
if isinstance(preds, dict):
    # 可能是 {qid: ans}
    print("Keys sample:", list(itertools.islice(preds.keys(), 5)))
    items = list(preds.items())[:5]
    print("Sample items:", items)
elif isinstance(preds, list):
    print("Len:", len(preds))
    for x in preds[:5]:
        print(x)
else:
    print("Unexpected preds type")


Type: <class 'list'>
Len: 5
{'question_id': 363652002, 'answer': 'The fire fighting implement in the image is a fire hydrant.'}
{'question_id': 278449003, 'answer': 'The pizza is round in shape.'}
{'question_id': 396676005, 'answer': 'The arrow points to the right.'}
{'question_id': 381832001, 'answer': 'The streetlamp is red.'}
{'question_id': 302964001, 'answer': 'This is a large red passenger bus.'}


In [ ]:
# ==== ONE-CELL: 20条 -> 绝对路径 -> 推理 -> 按VQA评分 ====
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
MODEL_ID = "/content/drive/MyDrive/models/llava-1.5-7b-hf"

# 1) 看目录
!ls -lah "{DATA_DIR}"

# 2) 取前20行做子集
!head -n 20 "{DATA_DIR}/inference.jsonl" > "{DATA_DIR}/inference_20.jsonl"
!wc -l "{DATA_DIR}/inference_20.jsonl"

# 3) 将 image 路径改为绝对路径（避免相对路径导致读图失败）
import json, os
src = f"{DATA_DIR}/inference_20.jsonl"
dst = f"{DATA_DIR}/inference_20_abs.jsonl"
missing = 0
with open(src, "r", encoding="utf-8") as f_in, open(dst, "w", encoding="utf-8") as f_out:
    for line in f_in:
        if not line.strip():
            continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path")
        if img and not img.startswith("/"):
            img = os.path.join(DATA_DIR, img)
        obj["image"] = img
        if not os.path.exists(img):
            missing += 1
        f_out.write(json.dumps(obj, ensure_ascii=False) + "\n")
print("Wrote:", dst, "| missing_images:", missing)

# 4) 推理（保留原先4bit用法）
!python /content/run_llava_infer.py \
  --in_jsonl "{DATA_DIR}/inference_20_abs.jsonl" \
  --out_preds "{DATA_DIR}/my_preds_20.json" \
  --model_id "{MODEL_ID}" \
  --load_in_4bit

# 5) 按 VQA 规则评分（score_preds.py 自带规范化与VQA打分）
!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_20.json"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 2.3M
drwx------ 2 root root 4.0K Aug  8 00:00 images
-rw------- 1 root root  622 Aug  8 00:25 inference_5.jsonl
-rw------- 1 root root 240K Aug  7 23:10 inference.jsonl
-rw------- 1 root root 319K Aug  7 23:10 manifest.csv
-rw------- 1 root root 1.6M Aug  7 22:47 mini_val_annotations.json
-rw------- 1 root root 185K Aug  7 22:47 mini_val_questions.json
-rw------- 1 root root  383 Aug  8 00:27 my_preds_5.json
20 /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_20.jsonl
Wrote: /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_20_abs.jsonl | missing_images: 0
2025-08-08 00:45:12.732658: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENAB

In [ ]:
# ==== ONE-CELL: 20条子集 -> 推理 -> 评分（含裁短后重评） ====
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
MODEL_ID = "/content/drive/MyDrive/models/llava-1.5-7b-hf"

# 1) 看看数据目录
!ls -lah "{DATA_DIR}"

# 2) 取前20条做子集
!head -n 20 "{DATA_DIR}/inference.jsonl" > "{DATA_DIR}/inference_20.jsonl"
!wc -l "{DATA_DIR}/inference_20.jsonl"

# 3) 把 images/... 改为绝对路径，避免当前目录不一致导致读图失败
import json, os
src = f"{DATA_DIR}/inference_20.jsonl"
dst = f"{DATA_DIR}/inference_20_abs.jsonl"
with open(src, "r", encoding="utf-8") as f_in, open(dst, "w", encoding="utf-8") as f_out:
    for line in f_in:
        if not line.strip():
            continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path")
        if img and not img.startswith("/"):
            obj["image"] = os.path.join(DATA_DIR, img)
        f_out.write(json.dumps(obj, ensure_ascii=False) + "\n")
print("Wrote:", dst)

# 4) 推理（保留你原本的4bit参数）
!python /content/run_llava_infer.py \
  --in_jsonl "{DATA_DIR}/inference_20_abs.jsonl" \
  --out_preds "{DATA_DIR}/my_preds_20.json" \
  --model_id "{MODEL_ID}" \
  --load_in_4bit

# 5) 原始预测直接评分
!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_20.json"

# 6) （推荐）把整句答案裁成短答案，再重评一次
import re
def _basic_norm(s:str)->str:
    s = s.strip().lower()
    s = re.sub(r"[^0-9a-z\s]", " ", s)
    s = re.sub(r"\b(?:a|an|the)\b", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def _to_short(ans:str)->str:
    a = _basic_norm(ans)
    # 常见句式提要：保留系动词/描述后面的核心片段
    a = re.sub(r".*\b(?:is|are|was|were|looks like|points to|pointing to)\b\s+", "", a)
    toks = a.split()
    if not toks: return a
    return toks[-1] if len(toks)==1 else " ".join(toks[-2:])

import json
src_pred = f"{DATA_DIR}/my_preds_20.json"
dst_pred = f"{DATA_DIR}/my_preds_20_short.json"
preds = json.load(open(src_pred,"r",encoding="utf-8"))
out = []
if isinstance(preds, dict):
    for qid, ans in preds.items():
        out.append({"question_id": int(qid), "answer": _to_short(str(ans))})
else:
    for item in preds:
        qid = int(item["question_id"])
        raw = item.get("answer") or item.get("pred") or item.get("text") or ""
        out.append({"question_id": qid, "answer": _to_short(str(raw))})
json.dump(out, open(dst_pred,"w",encoding="utf-8"))
print("Wrote:", dst_pred, "size:", len(out))

!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_20_short.json"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 2.3M
drwx------ 2 root root 4.0K Aug  8 00:00 images
-rw------- 1 root root 3.5K Aug  8 00:45 inference_20_abs.jsonl
-rw------- 1 root root 2.5K Aug  8 00:45 inference_20.jsonl
-rw------- 1 root root  622 Aug  8 00:25 inference_5.jsonl
-rw------- 1 root root 240K Aug  7 23:10 inference.jsonl
-rw------- 1 root root 319K Aug  7 23:10 manifest.csv
-rw------- 1 root root 1.6M Aug  7 22:47 mini_val_annotations.json
-rw------- 1 root root 185K Aug  7 22:47 mini_val_questions.json
-rw------- 1 root root 1.9K Aug  8 00:47 my_preds_20.json
-rw------- 1 root root  383 Aug  8 00:27 my_preds_5.json
20 /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_20.jsonl
Wrote: /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_20_abs.jsonl
2025-08-08 00:47:52.766200: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slig

In [ ]:
# ==== ONE-CELL: 20条 -> 绝对路径 + 短答指令 -> 推理 -> VQA评分（无后处理） ====
from google.colab import drive
drive.mount('/content/drive')

import os, json

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
MODEL_ID = "/content/drive/MyDrive/models/llava-1.5-7b-hf"

# 1) 取前20行到子集
src_all = f"{DATA_DIR}/inference.jsonl"
sub_abs = f"{DATA_DIR}/inference_20_abs_instr.jsonl"

INSTR = (
    "Answer with a single word or a very short phrase only. "
    "No punctuation or extra words.\nQ: {q}\nA:"
)

n, missing = 0, 0
with open(src_all, "r", encoding="utf-8") as fin, open(sub_abs, "w", encoding="utf-8") as fout:
    for line in fin:
        if not line.strip():
            continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path")
        if img and not img.startswith("/"):
            img = os.path.join(DATA_DIR, img)
        obj["image"] = img
        # 注入短答指令到 question（不改评测规则，只改变提示）
        q = obj.get("question","")
        obj["question"] = INSTR.format(q=q)
        if not os.path.exists(img):
            missing += 1
        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
        n += 1
        if n >= 20: break

print("Wrote:", sub_abs, "| samples:", n, "| missing_images:", missing)
!head -n 2 "{sub_abs}"

# 2) 推理（保持你的4bit加载）
!python /content/run_llava_infer.py \
  --in_jsonl "{sub_abs}" \
  --out_preds "{DATA_DIR}/my_preds_20_instr.json" \
  --model_id "{MODEL_ID}" \
  --load_in_4bit

# 3) 评分（严格按 VQA 规则；score_preds.py 自带规范化）
!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_20_instr.json"

# 4) 可选：打印前5条预测，方便人工抽查（不影响评分）
try:
    preds = json.load(open(f"{DATA_DIR}/my_preds_20_instr.json","r",encoding="utf-8"))
    print("SAMPLE PREDS:", preds[:5] if isinstance(preds, list) else list(list(preds.items())[:5]))
except Exception as e:
    print("Preview preds failed:", e)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Wrote: /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_20_abs_instr.jsonl | samples: 20 | missing_images: 0
{"question_id": 363652002, "image": "/content/drive/MyDrive/baseline_test/subset_vqa_2000/images/COCO_val2014_000000363652.jpg", "question": "Answer with a single word or a very short phrase only. No punctuation or extra words.\nQ: What is the fire fighting implement called?\nA:"}
{"question_id": 278449003, "image": "/content/drive/MyDrive/baseline_test/subset_vqa_2000/images/COCO_val2014_000000278449.jpg", "question": "Answer with a single word or a very short phrase only. No punctuation or extra words.\nQ: What is the shape of the pizza?\nA:"}
2025-08-08 00:58:33.624269: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from differ

In [ ]:
# ==== ONE-CELL: 2000条 -> 绝对路径 + 短答指令 -> 推理 -> VQA评分 ====
from google.colab import drive
drive.mount('/content/drive')

import os, json, time

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
MODEL_ID = "/content/drive/MyDrive/models/llava-1.5-7b-hf"

# 1) 生成带绝对路径 + 短答指令的 JSONL（覆盖全量 2000 条）
src_all = f"{DATA_DIR}/inference.jsonl"
dst_abs = f"{DATA_DIR}/inference_2000_abs_instr.jsonl"

INSTR = (
    "Answer with a single word or a very short phrase only. "
    "No punctuation or extra words.\nQ: {q}\nA:"
)

n, missing, t0 = 0, 0, time.time()
with open(src_all, "r", encoding="utf-8") as fin, open(dst_abs, "w", encoding="utf-8") as fout:
    for line in fin:
        if not line.strip():
            continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path")
        if img and not img.startswith("/"):
            img = os.path.join(DATA_DIR, img)
        obj["image"] = img
        # 注入短答指令（不影响评分，因为评分只用 question_id/answers）
        q = obj.get("question","")
        obj["question"] = INSTR.format(q=q)
        if not os.path.exists(img):
            missing += 1
        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
        n += 1
prep_sec = round(time.time()-t0, 1)
print(f"Prepared: {dst_abs} | samples: {n} | missing_images: {missing} | prep_sec: {prep_sec}")
!wc -l "{dst_abs}"
!head -n 2 "{dst_abs}"

# 2) 推理（注意：2000条可能需要较长时间）
t0 = time.time()
!python /content/run_llava_infer.py \
  --in_jsonl "{dst_abs}" \
  --out_preds "{DATA_DIR}/my_preds_2000_instr.json" \
  --model_id "{MODEL_ID}" \
  --load_in_4bit
print("Inference sec:", round(time.time()-t0, 1))

# 3) 按 VQA 规则评分（score_preds.py 已内置规范化与VQA打分）
!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{DATA_DIR}/my_preds_2000_instr.json" \
  --report_csv "{DATA_DIR}/report_2000_instr.csv"

# 4) 预览前5条预测（方便抽查）
try:
    preds_path = f"{DATA_DIR}/my_preds_2000_instr.json"
    preds = json.load(open(preds_path, "r", encoding="utf-8"))
    preview = preds if isinstance(preds, list) else [{"question_id": int(k), "answer": v} for k,v in list(preds.items())[:5]]
    print("SAMPLE PREDS:", preview[:5])
    print("Preds file:", preds_path)
    print("Report CSV:", f"{DATA_DIR}/report_2000_instr.csv")
except Exception as e:
    print("Preview preds failed:", e)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Prepared: /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_2000_abs_instr.jsonl | samples: 2000 | missing_images: 0 | prep_sec: 0.3
2000 /content/drive/MyDrive/baseline_test/subset_vqa_2000/inference_2000_abs_instr.jsonl
{"question_id": 363652002, "image": "/content/drive/MyDrive/baseline_test/subset_vqa_2000/images/COCO_val2014_000000363652.jpg", "question": "Answer with a single word or a very short phrase only. No punctuation or extra words.\nQ: What is the fire fighting implement called?\nA:"}
{"question_id": 278449003, "image": "/content/drive/MyDrive/baseline_test/subset_vqa_2000/images/COCO_val2014_000000278449.jpg", "question": "Answer with a single word or a very short phrase only. No punctuation or extra words.\nQ: What is the shape of the pizza?\nA:"}
2025-08-08 01:06:07.814966: I tensorflow/core/util/port.cc:153] oneDNN custom operat

In [ ]:
# ==== ONE-CELL: 基于已有预测，导出逐样本明细 + 分类型统计 ====
from google.colab import drive
drive.mount('/content/drive')

import os, json, csv, re, collections

DATA_DIR = "/content/drive/MyDrive/baseline_test/subset_vqa_2000"
paths = {
    "ann":  os.path.join(DATA_DIR, "mini_val_annotations.json"),
    "inf":  os.path.join(DATA_DIR, "inference.jsonl"),                 # 原始问题（无指令）
    "pred": os.path.join(DATA_DIR, "my_preds_2000_instr.json"),        # 你刚跑出的预测
    "per_sample": os.path.join(DATA_DIR, "eval_per_sample_2000.csv"),
    "by_type":    os.path.join(DATA_DIR, "breakdown_by_type_2000.csv"),
    "by_qtype":   os.path.join(DATA_DIR, "breakdown_by_qtype_2000.csv"),
    "errors":     os.path.join(DATA_DIR, "errors_2000.csv"),
}

# --- 与 score_preds.py 保持一致的规范化与打分 ---
NUM_WORDS = {
    "zero":"0","one":"1","two":"2","three":"3","four":"4","five":"5",
    "six":"6","seven":"7","eight":"8","nine":"9","ten":"10","eleven":"11",
    "twelve":"12","thirteen":"13","fourteen":"14","fifteen":"15",
    "sixteen":"16","seventeen":"17","eighteen":"18","nineteen":"19","twenty":"20"
}
ARTICLES = {"a","an","the"}

def normalize_ans(a: str) -> str:
    a = a.strip().lower()
    a = re.sub(r"[^0-9a-z\s]", " ", a)
    toks = [t for t in a.split() if t and t not in ARTICLES]
    toks = [NUM_WORDS.get(w, w) for w in toks]
    a = " ".join(toks)
    a = re.sub(r"\s+", " ", a).strip()
    return a

def vqa_score(pred_norm: str, answers_list):
    # answers_list: [{"answer": "..."}] * 10
    n_match = 0
    for a in answers_list:
        gt = normalize_ans(str(a.get("answer","")))
        if gt == pred_norm:
            n_match += 1
    return min(1.0, n_match/3.0), n_match

# --- 载入数据 ---
# 标注（含 answers/question_type/answer_type）
ann = json.load(open(paths["ann"], "r", encoding="utf-8"))
ann_items = ann["annotations"] if isinstance(ann, dict) else ann
ann_by_qid = {int(x["question_id"]): x for x in ann_items}

# 问题与图片路径（用原始 inference.jsonl，避免把“短答指令”混进去）
qimg_by_qid = {}
with open(paths["inf"], "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip(): continue
        obj = json.loads(line)
        qid = int(obj["question_id"])
        q = obj.get("question","")
        img = obj.get("image") or obj.get("image_path") or ""
        # 补绝对路径，便于之后定位图片
        if img and not img.startswith("/"):
            img_abs = os.path.join(DATA_DIR, img)
        else:
            img_abs = img
        qimg_by_qid[qid] = {"question": q, "image": img, "image_abs": img_abs}

# 预测
pred = json.load(open(paths["pred"], "r", encoding="utf-8"))
if isinstance(pred, dict):
    preds = [{"question_id": int(k), "answer": v} for k,v in pred.items()]
else:
    preds = [{"question_id": int(x["question_id"]), "answer": x.get("answer") or x.get("pred") or x.get("text") or ""} for x in pred]

# --- 逐样本评估并导出 CSV ---
per_rows = []
by_type_sum = collections.defaultdict(lambda: {"n":0, "s":0.0, "full":0})
by_qtype_sum = collections.defaultdict(lambda: {"n":0, "s":0.0, "full":0})
n_total, s_total, n_full = 0, 0.0, 0

for p in preds:
    qid = int(p["question_id"])
    ans_raw = str(p["answer"])
    ans_norm = normalize_ans(ans_raw)

    ann_item = ann_by_qid.get(qid)
    if not ann_item:
        # 跳过不在标注里的 qid（理论上不会发生）
        continue

    answers_list = ann_item.get("answers") or []
    # 兜底：如果没有 answers，但有 multiple_choice_answer，则复制成10个（你的标注应当已有 answers）
    if not answers_list and ann_item.get("multiple_choice_answer"):
        answers_list = [{"answer": ann_item["multiple_choice_answer"]} for _ in range(10)]

    score, n_match = vqa_score(ans_norm, answers_list)
    n_total += 1; s_total += score
    if score >= 1.0 - 1e-9: n_full += 1

    atype = ann_item.get("answer_type","other")
    qtype = ann_item.get("question_type","")

    by_type_sum[atype]["n"] += 1
    by_type_sum[atype]["s"] += score
    if score >= 1.0 - 1e-9: by_type_sum[atype]["full"] += 1

    by_qtype_sum[qtype]["n"] += 1
    by_qtype_sum[qtype]["s"] += score
    if score >= 1.0 - 1e-9: by_qtype_sum[qtype]["full"] += 1

    # 统计 GT top3（按规范化聚合）
    counter = collections.Counter(normalize_ans(a.get("answer","")) for a in answers_list)
    gt_top3 = "; ".join([f"{k}:{v}" for k,v in counter.most_common(3)])

    qinfo = qimg_by_qid.get(qid, {"question":"", "image":"", "image_abs":""})
    per_rows.append({
        "question_id": qid,
        "image": qinfo["image"],
        "image_abs": qinfo["image_abs"],
        "question": qinfo["question"],
        "pred_answer_raw": ans_raw,
        "pred_answer_norm": ans_norm,
        "gt_top3_norm:count": gt_top3,
        "match_count": n_match,
        "vqa_score": round(score, 4),
        "answer_type": atype,
        "question_type": qtype,
    })

# 保存逐样本明细
fieldnames = list(per_rows[0].keys()) if per_rows else []
with open(paths["per_sample"], "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader(); w.writerows(per_rows)

# 保存错误样本（分数<1）
with open(paths["errors"], "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    for r in per_rows:
        if r["vqa_score"] < 1.0:
            w.writerow(r)

# 分类型统计（answer_type）
with open(paths["by_type"], "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["answer_type","n","acc_mean","full_rate"])
    w.writeheader()
    for k, v in sorted(by_type_sum.items()):
        n = v["n"]; acc = (v["s"]/n) if n else 0.0; full = (v["full"]/n) if n else 0.0
        w.writerow({"answer_type": k, "n": n, "acc_mean": round(acc,4), "full_rate": round(full,4)})

# 分 question_type 统计（可选）
with open(paths["by_qtype"], "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["question_type","n","acc_mean","full_rate"])
    w.writeheader()
    for k, v in sorted(by_qtype_sum.items(), key=lambda x:x[0]):
        n = v["n"]; acc = (v["s"]/n) if n else 0.0; full = (v["full"]/n) if n else 0.0
        w.writerow({"question_type": k, "n": n, "acc_mean": round(acc,4), "full_rate": round(full,4)})

overall = (s_total/n_total) if n_total else 0.0
print("Saved:")
for k, p in paths.items():
    if k in ("ann","inf","pred"): continue
    print(f"- {k}: {p}")
print(f"Overall (recomputed here) acc_mean={overall:.4f}  full_rate={(n_full/n_total if n_total else 0):.4f}  samples={n_total}")


Mounted at /content/drive
Saved:
- per_sample: /content/drive/MyDrive/baseline_test/subset_vqa_2000/eval_per_sample_2000.csv
- by_type: /content/drive/MyDrive/baseline_test/subset_vqa_2000/breakdown_by_type_2000.csv
- by_qtype: /content/drive/MyDrive/baseline_test/subset_vqa_2000/breakdown_by_qtype_2000.csv
- errors: /content/drive/MyDrive/baseline_test/subset_vqa_2000/errors_2000.csv
Overall (recomputed here) acc_mean=0.7607  full_rate=0.7000  samples=2000


In [ ]:
%%writefile /content/run_llava_infer.py
# -*- coding: utf-8 -*-
import os, json, argparse, sys
from PIL import Image
from tqdm import tqdm
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--in_jsonl", required=True)
    ap.add_argument("--out_preds", required=True)
    ap.add_argument("--model_id", default="llava-hf/llava-1.5-7b-hf")
    ap.add_argument("--max_new_tokens", type=int, default=12)  # VQA 短答足够
    ap.add_argument("--load_in_4bit", action="store_true")
    ap.add_argument("--save_every", type=int, default=200, help="flush every N samples")
    args = ap.parse_args()

    # ===== 4bit 量化（新写法） =====
    quantization_config = None
    if args.load_in_4bit:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,  # 比 float32 更快
        )

    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model = LlavaForConditionalGeneration.from_pretrained(
        args.model_id,
        device_map="auto",
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=True,
        quantization_config=quantization_config,  # 用新接口，不再传 load_in_4bit
    )
    processor = AutoProcessor.from_pretrained(args.model_id, use_fast=True)
    model.eval()
    torch.set_grad_enabled(False)

    # ===== 断点续跑：读取已完成的预测 =====
    os.makedirs(os.path.dirname(args.out_preds), exist_ok=True)
    tmp_jsonl = args.out_preds + ".tmp.jsonl"   # 流式增量文件
    pred_list = []
    done_qids = set()

    if os.path.exists(tmp_jsonl):
        with open(tmp_jsonl, "r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                rec = json.loads(line)
                pred_list.append(rec)
                qid = int(rec.get("question_id"))
                done_qids.add(qid)
        print(f"[resume] loaded {len(pred_list)} records from {tmp_jsonl}", flush=True)

    fout = open(tmp_jsonl, "a", encoding="utf-8")

    def flush_snapshot():
        """把 list 快照保存到 out_preds（你的评分脚本可直接用）"""
        snap = args.out_preds + ".partial"
        with open(snap, "w", encoding="utf-8") as f:
            json.dump(pred_list, f, ensure_ascii=False)
        os.replace(snap, args.out_preds)
        fout.flush()
        try:
            os.fsync(fout.fileno())
        except Exception:
            pass

    # ===== 读取输入 =====
    root = os.path.dirname(args.in_jsonl)
    with open(args.in_jsonl, "r", encoding="utf-8") as f:
        lines = [json.loads(x) for x in f if x.strip()]

    # ===== 生成参数：保守解码，稳定短答 =====
    gen_kwargs = dict(
        do_sample=False,
        temperature=0.0,
        top_p=1.0,
        num_beams=1,
        max_new_tokens=args.max_new_tokens,
        repetition_penalty=1.0,
    )

    pbar = tqdm(lines, total=len(lines), ncols=0)
    for r in pbar:
        qid = int(r["question_id"])
        if qid in done_qids:
            continue

        img_path = r["image"]
        if not os.path.isabs(img_path):
            img_path = os.path.join(root, img_path)

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # 若图片损坏/缺失，给个占位答案，避免整个进程中断
            pred_list.append({"question_id": qid, "answer": ""})
            fout.write(json.dumps({"question_id": qid, "answer": ""}, ensure_ascii=False) + "\n")
            if len(pred_list) % max(1, args.save_every) == 0:
                flush_snapshot()
            continue

        question = r["question"]

        # LLaVA 对话模板
        convo = [{"role":"user","content":[{"type":"image"}, {"type":"text","text":question}]}]
        prompt = processor.apply_chat_template(convo, add_generation_prompt=True)

        inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            out = model.generate(**inputs, **gen_kwargs)

        text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
        # 和你原来一致：截取 ASSISTANT: 之后
        ans = text.split("ASSISTANT:")[-1].strip()

        rec = {"question_id": qid, "answer": ans}
        pred_list.append(rec)
        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")

        if len(pred_list) % max(1, args.save_every) == 0:
            pbar.set_description(f"flushing @ {len(pred_list)}")
            flush_snapshot()

    # 最后一次完整保存
    flush_snapshot()
    fout.close()
    print("Saved:", args.out_preds)

if __name__ == "__main__":
    main()


Writing /content/run_llava_infer.py


In [ ]:
# ==== FIX ENV + RUN 10K ====
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, sys, json, time, importlib

# 1) 重写 score_preds.py（和你之前的一致）
score_code = r'''
# -*- coding: utf-8 -*-
import json, re, argparse, csv, collections
NUM_WORDS = {"zero":"0","one":"1","two":"2","three":"3","four":"4","five":"5","six":"6","seven":"7","eight":"8","nine":"9","ten":"10","eleven":"11","twelve":"12","thirteen":"13","fourteen":"14","fifteen":"15","sixteen":"16","seventeen":"17","eighteen":"18","nineteen":"19","twenty":"20"}
ARTICLES = {"a","an","the"}
def normalize(a):
    a = a.strip().lower(); a = re.sub(r"[^\w\s]", " ", a)
    toks = [w for w in a.split() if w not in ARTICLES]
    toks = [NUM_WORDS.get(w, w) for w in toks]
    return re.sub(r"\s+"," "," ".join(toks)).strip()
def score(pred_norm, answers):
    m = sum(1 for x in answers if normalize(x.get("answer","")) == pred_norm)
    return min(1.0, m/3.0)
def load_preds(p):
    obj = json.load(open(p,"r",encoding="utf-8"))
    return ({int(x["question_id"]): x["answer"] for x in obj} if isinstance(obj,list)
            else {int(k): v for k,v in obj.items()})
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--mini_a", required=True)
    ap.add_argument("--preds", required=True)
    ap.add_argument("--report_csv", default=None)
    a = ap.parse_args()
    ann = json.load(open(a.mini_a,"r",encoding="utf-8"))
    ann_by_qid = {x["question_id"]: x for x in ann["annotations"]}
    preds = load_preds(a.preds)
    tot,s = 0,0.0; bytype = collections.defaultdict(lambda: {"n":0,"s":0.0})
    for qid, ans in preds.items():
        if qid not in ann_by_qid: continue
        acc = score(normalize(ans), ann_by_qid[qid].get("answers",[]))
        s += acc; tot += 1
        t = ann_by_qid[qid].get("answer_type","other")
        bytype[t]["n"] += 1; bytype[t]["s"] += acc
    overall = s/tot if tot else 0.0
    print(f"samples={tot}  overall_acc={overall:.4f}")
    for k,v in bytype.items():
        print(f"acc_{k}={(v['s']/v['n'] if v['n'] else 0.0):.4f}")
    if a.report_csv:
        row = {"overall_acc": overall, "samples": tot}
        for k,v in bytype.items(): row[f"acc_{k}"] = v["s"]/v["n"] if v["n"] else 0.0
        with open(a.report_csv,"w",newline="",encoding="utf-8") as f:
            import csv; w = csv.DictWriter(f, fieldnames=row.keys()); w.writeheader(); w.writerow(row)
if __name__ == "__main__":
    main()
'''
with open('/content/score_preds.py','w',encoding='utf-8') as f:
    f.write(score_code)
print("Wrote /content/score_preds.py")

# 2) 路径配置（按你的 10k 目录）
DATA_DIR  = "/content/drive/MyDrive/baseline_test/subset_vqa_10000"
MODEL_ID  = "/content/drive/MyDrive/models/llava-1.5-7b-hf"
IN_JSONL  = f"{DATA_DIR}/inference.jsonl"
DST_JSONL = f"{DATA_DIR}/inference_10000_abs_instr.jsonl"
PREDS     = f"{DATA_DIR}/my_preds_10000_instr.json"
REPORT    = f"{DATA_DIR}/report_10000_instr.csv"

# 确认推理脚本存在（/content 是易失）
assert os.path.exists("/content/run_llava_infer.py"), "缺 /content/run_llava_infer.py，请先再写入一遍。"

# 3) 生成带绝对路径 + 短答指令的 JSONL（如果已存在可跳过这步）
INSTR = ("Answer with a single word or a very short phrase only. "
         "No punctuation or extra words.\nQ: {q}\nA:")

n, missing, t0 = 0, 0, time.time()
with open(IN_JSONL, "r", encoding="utf-8") as fin, open(DST_JSONL, "w", encoding="utf-8") as fout:
    for line in fin:
        if not line.strip(): continue
        obj = json.loads(line)
        img = obj.get("image") or obj.get("image_path")
        if img and not img.startswith("/"):
            img = os.path.join(DATA_DIR, img)
        obj["image"] = img
        obj["question"] = INSTR.format(q=obj.get("question",""))
        if not os.path.exists(img):
            missing += 1
        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
        n += 1
print(f"Prepared: {DST_JSONL} | samples: {n} | missing_images: {missing} | prep_sec: {round(time.time()-t0,1)}")
assert missing == 0, f"有 {missing} 张图片缺失，先补齐再跑。"

# 4) 推理（10k 会很久；--save_every 支持断点续跑）
t0 = time.time()
!python /content/run_llava_infer.py \
  --in_jsonl "{DST_JSONL}" \
  --out_preds "{PREDS}" \
  --model_id "{MODEL_ID}" \
  --save_every 200
print("Inference sec:", round(time.time()-t0, 1))

# 5) 评分
!python /content/score_preds.py \
  --mini_a "{DATA_DIR}/mini_val_annotations.json" \
  --preds  "{PREDS}" \
  --report_csv "{REPORT}"

# 6) 预览预测
try:
    preds = json.load(open(PREDS, "r", encoding="utf-8"))
    preview = preds if isinstance(preds, list) else [{"question_id": int(k), "answer": v} for k,v in list(preds.items())[:5]]
    print("SAMPLE PREDS:", preview[:5])
    print("Preds file:", PREDS)
    print("Report CSV:", REPORT)
except Exception as e:
    print("Preview preds failed:", e)


Streaming output truncated to the last 5000 lines.
flushing @ 1200:  13% 1253/10000 [20:27<2:27:30,  1.01s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
flushing @ 1200:  13% 1254/10000 [20:28<2:35:50,  1.07s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
flushing @ 1200:  13% 1255/10000 [20:29<2:23:25,  1.02it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
flushing @ 1200:  13% 1256/10000 [20:30<2:22:51,  1.02it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
flushing @ 1200:  13% 1257/10000 [20:31<2:18:40,  1.05it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` fo